In [22]:
from datasets import load_dataset
train_dataset = load_dataset("imdb", split="train").to_pandas()
test_dataset = load_dataset("imdb", split="test").to_pandas()

Reusing dataset imdb (C:\Users\PION\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (C:\Users\PION\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [2]:
test_dataset

,text,label
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
...,...,...
24995,I occasionally let my kids watch this garbage ...,0
24996,When all we have anymore is pretty much realit...,0
24997,The basic genre is a thriller intercut with an...,0
24998,Four things intrigued me as to this film - fir...,0


In [5]:
test_label = test_dataset.groupby('label')
test_label.count()

,text
label,
0,12500
1,12500


In [6]:
train_label = train_dataset.groupby('label')
train_label.count()

,text
label,
0,12500
1,12500


In [7]:
test_dataset[test_dataset['label'] == 0].count()

text     12500
label    12500
dtype: int64

In [8]:
train_dataset[train_dataset['label'] == 0].count()

text     12500
label    12500
dtype: int64

In [9]:
train_dataset['text'] = train_dataset['text'].str.lower()
test_dataset['text'] = test_dataset['text'].str.lower()
test_dataset

,text,label
0,i went and saw this movie last night after bei...,1
1,actor turned director bill paxton follows up h...,1
2,as a recreational golfer with some knowledge o...,1
3,"i saw this film in a sneak preview, and it is ...",1
4,bill paxton has taken the true story of the 19...,1
...,...,...
24995,i occasionally let my kids watch this garbage ...,0
24996,when all we have anymore is pretty much realit...,0
24997,the basic genre is a thriller intercut with an...,0
24998,four things intrigued me as to this film - fir...,0


In [10]:
from string import punctuation
train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset
#Cleaning dataset (part 1)

C:\Users\PION\AppData\Local\Temp\ipykernel_6192\311460489.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
C:\Users\PION\AppData\Local\Temp\ipykernel_6192\311460489.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')


,text,label
0,i went and saw this movie last night after bei...,1
1,actor turned director bill paxton follows up h...,1
2,as a recreational golfer with some knowledge o...,1
3,i saw this film in a sneak preview and it is d...,1
4,bill paxton has taken the true story of the 19...,1
...,...,...
24995,i occasionally let my kids watch this garbage ...,0
24996,when all we have anymore is pretty much realit...,0
24997,the basic genre is a thriller intercut with an...,0
24998,four things intrigued me as to this film firs...,0


In [13]:
#Creating our model
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#TfidfVectorizer is capable of doing both word count vectorization and IDF calculations.
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.metrics import accuracy_score
import numpy as np

C:\Users\PION\anaconda3\envs\PYTORCH\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
text_pip = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB()) ])
text_pip.fit(train_dataset['text'],train_dataset['label'])
pred = text_pip.predict(test_dataset['text'])

In [18]:
metrics.confusion_matrix(test_dataset['label'], pred)

array([[10989,  1511],
       [ 3057,  9443]], dtype=int64)

In [21]:
print('Accuracy is ' + str(np.mean(pred == test_dataset.label)))
#Pas compris si je dois tester aussi sur les données d'entrainement

Accuracy is 0.81728


## Why is accuracy a sufficient measure of evaluation here?
#### L'accuracy est une bonne mesure dans notre cas puisque notre dataset est bien balance. Chaque type de label possede un nobre similaire de donnée permettant une meilleur representation
